In [1]:
# Import libs
import findspark
findspark.init("/home/antonis/spark-2.3.0-bin-hadoop2.7")

import os.path
import pandas
import math
import time

from metrics.Correlation import Correlation
from IO.Write import Write
from IO.Read import Read
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet

import pyspark
from pyspark.sql import SparkSession


import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)
import DataSet.SnpsSelection as s

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [2]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "20g")
       .set('spark_executor_cores',"3"))

In [3]:
#spark = SparkSession.builder.appName('melanoma').getOrCreate()
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [ ]:
#spark.stop()

In [ ]:
#path = '/home/george/diplwmatikh/code/maf/maf = 0.05/assocs/'
#pathPatients = '/home/george/diplwmatikh/data/patients/Patients.txt'
pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snp1.txt'

# Read ASSOCS

In [ ]:

start_time = time.time()
#path = '/home/george/snp/'
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
log_txt = sc.textFile(path)
sc.setLogLevel("WARN")
header = log_txt.first()
log_txt = log_txt.filter(lambda line: line != header)
temp_var = log_txt.map(lambda k: k.split())

inferRow = temp_var.map(lambda p: Row(CHR=(p[0].strip()),SNP=p[1].strip(),BP=p[2].strip(),A1=p[3].strip(),F_A=p[4].strip()
                                   ,F_U=p[5].strip(),A2=p[6].strip(),CHISQ=p[7].strip(),P=(p[8].strip())
                                      ,OR=p[9].strip()))
schemaString = "CHR SNP BP A1 F_A F_U A2 CHISQ P OR"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
schemaAssocs = spark.createDataFrame(inferRow,schema)
schemaAssocs.createOrReplaceTempView("Assocs")
#df=temp_var.toDF(header.split())
#log_df.select(log_df['field2']).show()
#log_df.filter(log_df['field2']=='1').count()
elapsed_time = time.time() - start_time
print(elapsed_time)
#df =spark.read.csv(path,header=True,inferSchema=True,escape="\t")

# Read Patients

In [ ]:
start_time = time.time()
#path = '/home/george/snp/'
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
log_txt = sc.textFile(pathPatients)
sc.setLogLevel("WARN")
header = log_txt.first()
log_txt = log_txt.filter(lambda line: line != header)
temp_var = log_txt.map(lambda k: k.split())

inferRow = temp_var.map(lambda p: Row(eid=(p[0].strip()),sex=p[1].strip(),birth_year=p[2].strip(),cases=p[3].strip()))
schemaString = "eid sex birth_year cases"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
schemaPatients = spark.createDataFrame(inferRow,schema)
schemaPatients.createOrReplaceTempView("Patients")
#df=temp_var.toDF(header.split())
#log_df.select(log_df['field2']).show()
#log_df.filter(log_df['field2']=='1').count()
elapsed_time = time.time() - start_time
print(elapsed_time)
#df =spark.read.csv(path,header=True,inferSchema=True,escape="\t")

# TO REMOVE NA

In [ ]:
schemaAssocs = schemaAssocs.where("OR != 'NA'").drop()

In [ ]:
schemaAssocs.createOrReplaceTempView("Assocs")


In [ ]:
sqlDF = spark.sql("SELECT * FROM Assocs WHERE SNP=='.' ")
sqlDF.show()

In [ ]:
schemaPatients.show()

# STRING INDEXING

In [ ]:
snp_indexer = StringIndexer(inputCol='SNP',outputCol='SNPIndex')
id_indexer = StringIndexer(inputCol='eid',outputCol='eidIndex')

In [ ]:
Patients =id_indexer.fit(schemaPatients).transform(schemaPatients)

In [ ]:
Patients.show()

In [ ]:
Assocs = snp_indexer.fit(schemaAssocs).transform(schemaAssocs)

In [ ]:
Assocs.show()

# GET IDS FROM PATINETS AND SNP

In [ ]:
def setIdToName(aList):
    
    ids = {}
    nameToId = {}
    idToName = {}
    count = 0
    
    for i in aList:
        
        nameToId[i] = count
        idToName[count] = i
        count += 1
        
    ids['nameToId'] = nameToId
    ids['idToName'] = idToName
    
    yield ids

In [ ]:
ids={}
test= Patients.select('eid')
#idss = test.toPandas().to_dict(orient='list')
snps = Assocs.select('SNPIndex')
idsa = snps.toPandas().to_dict(orient='list')
ids['patients'] = setIdToName(test.toPandas().to_dict(orient='list')['eid'])
ids['snps'] = idsa['SNPIndex']

In [ ]:
X, Y = read.readSnpsCode(patients,ids)

In [ ]:
test.toPandas().to_dict(orient='list')['eid']

# TESTS

In [35]:
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.01/snp2.txt'
pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'

#pathSnp = '/media/antonis/Antonis_Moulopoulos/newSet/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/Antonis_Moulopoulos/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'

data = spark.read.option("maxColumns", 80000).csv(pathSnp,inferSchema=True,header=True)
data=data.withColumnRenamed('TARGET','label')

In [36]:
features = []
for i in data.columns:
    if 'rs'  in i:
        features.append(i)
print(len(features))

12002


In [37]:
data.select('patients','label').show(10)

+--------+-----+
|patients|label|
+--------+-----+
| 3401706|    0|
| 1021783|    0|
| 3506859|    0|
| 5834548|    0|
| 3852084|    1|
| 5647755|    0|
| 1133041|    0|
| 4077867|    0|
| 2572353|    0|
| 4266292|    0|
+--------+-----+
only showing top 10 rows



In [7]:
'''from pyspark.ml.regression import AFTSurvivalRegression
from pyspark.ml.linalg import Vectors

training = spark.createDataFrame([
    ( 1.0, Vectors.dense(1.560, -0.605)),
    ( 0.0, Vectors.dense(0.346, 2.158)),
    ( 0.0, Vectors.dense(1.380, 0.231)),
    ( 1.0, Vectors.dense(0.520, 1.151)),
    ( 0.0, Vectors.dense(0.795, -0.226))], ["label", "features"])

training.show()'''

'from pyspark.ml.regression import AFTSurvivalRegression\nfrom pyspark.ml.linalg import Vectors\n\ntraining = spark.createDataFrame([\n    ( 1.0, Vectors.dense(1.560, -0.605)),\n    ( 0.0, Vectors.dense(0.346, 2.158)),\n    ( 0.0, Vectors.dense(1.380, 0.231)),\n    ( 1.0, Vectors.dense(0.520, 1.151)),\n    ( 0.0, Vectors.dense(0.795, -0.226))], ["label", "features"])\n\ntraining.show()'

In [38]:
assempler = VectorAssembler(inputCols=features,outputCol='features')
output = assempler.transform(data)
final_data = output.select('features','label')
#final_data.show(10)

In [39]:
final_data.show(10)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(12002,[5,6,8,9,1...|    0|
|(12002,[15,18,19,...|    0|
|(12002,[2,3,5,6,1...|    0|
|(12002,[2,3,6,11,...|    0|
|(12002,[1,2,4,6,7...|    1|
|(12002,[0,2,5,6,9...|    0|
|(12002,[1,2,4,7,1...|    0|
|(12002,[0,2,5,6,9...|    0|
|(12002,[0,1,2,11,...|    0|
|(12002,[0,1,2,3,6...|    0|
+--------------------+-----+
only showing top 10 rows



In [58]:
final_data.take(1)

[Row(features=SparseVector(12002, {5: 1.0, 6: 2.0, 8: 1.0, 9: 1.0, 10: 2.0, 15: 2.0, 16: 1.0, 17: 1.0, 24: 1.0, 40: 1.0, 41: 1.0, 44: 2.0, 47: 2.0, 53: 1.0, 60: 2.0, 62: 1.0, 63: 1.0, 65: 1.0, 66: 1.0, 68: 1.0, 69: 1.0, 70: 1.0, 71: 2.0, 72: 1.0, 75: 1.0, 79: 2.0, 81: 1.0, 83: 2.0, 86: 1.0, 92: 1.0, 93: 1.0, 94: 1.0, 98: 2.0, 102: 1.0, 104: 1.0, 105: 1.0, 108: 1.0, 117: 1.0, 118: 1.0, 125: 1.0, 126: 1.0, 128: 1.0, 130: 1.0, 131: 1.0, 141: 1.0, 145: 1.0, 146: 1.0, 148: 1.0, 152: 1.0, 157: 1.0, 158: 1.0, 159: 1.0, 160: 1.0, 166: 1.0, 173: 1.0, 174: 1.0, 179: 1.0, 181: 1.0, 183: 1.0, 184: 1.0, 189: 2.0, 194: 1.0, 196: 1.0, 198: 1.0, 204: 1.0, 206: 1.0, 207: 1.0, 209: 1.0, 218: 1.0, 219: 1.0, 220: 1.0, 221: 1.0, 226: 2.0, 228: 1.0, 231: 2.0, 232: 1.0, 235: 1.0, 237: 1.0, 238: 1.0, 241: 1.0, 244: 1.0, 246: 1.0, 254: 2.0, 257: 1.0, 258: 1.0, 260: 1.0, 262: 1.0, 264: 1.0, 266: 1.0, 267: 1.0, 268: 2.0, 269: 1.0, 272: 2.0, 274: 2.0, 278: 1.0, 283: 2.0, 286: 1.0, 289: 1.0, 292: 1.0, 294: 1.0, 29

# CORRELATION

In [11]:

from pyspark.ml.stat import Correlation
final_data1 = final_data2.select('features')
corr = Correlation.corr(final_data2, "features")
corr = corr.head()[0].toArray()


In [13]:
snpsRed = []
snpsRed = s.lowCorrelation(corr, threshold=0.7, up=100, down=99)


count =  9277
len snpsRed =  9277


# CHI-SQUARE


In [14]:
'''from pyspark.ml.stat import ChiSquareTest
from pyspark.ml.linalg import Vectors
data1 = [(1.0, Vectors.dense(23.0, 5.0)),
        (0.0, Vectors.dense(8.0, 10.0))]
df = spark.createDataFrame(data1, ["label", "features"])

r1 = ChiSquareTest.test(df, "features", "label").head()

#r = ChiSquareTest.test(final_data, "features", "label").head()
print("pValues: " + str(r1.pValues))
print("degreesOfFreedom: " + str(r1.degreesOfFreedom))
print("statistics: " + str(r1.statistics))'''

SyntaxError: invalid syntax (<ipython-input-14-18398d5610f2>, line 9)

In [15]:
'''count = 0 
j= 0 
for i in r[0].toArray():
    if i >= 0.05:
        print(features[j])
        count += 1
    j += 1
print("count = ",count)'''

'count = 0 \nj= 0 \nfor i in r[0].toArray():\n    if i >= 0.05:\n        print(features[j])\n        count += 1\n    j += 1\nprint("count = ",count)'

In [16]:
snpsRed1 = []
for i in snpsRed:
    snpsRed1.append(features[i])

assempler = VectorAssembler(inputCols=snpsRed1,outputCol='snpsRed')
output = assempler.transform(data)
final_data = output.select('snpsRed','label')
#final_data.show(10)

In [17]:
final_data = output.select('snpsRed','label')
final_data.show(10)

+--------------------+-----+
|             snpsRed|label|
+--------------------+-----+
|(9277,[5,6,8,9,10...|    0|
|(9277,[15,18,19,2...|    0|
|(9277,[2,3,5,6,10...|    0|
|(9277,[2,3,6,11,1...|    0|
|(9277,[1,2,4,6,7,...|    1|
|(9277,[0,2,5,6,9,...|    0|
|(9277,[1,2,4,7,10...|    0|
|(9277,[0,2,5,6,9,...|    0|
|(9277,[0,1,2,11,1...|    0|
|(9277,[0,1,2,3,6,...|    0|
+--------------------+-----+
only showing top 10 rows



In [18]:
#final_data = output.filter('label==1').select('snpsRed','label')

In [44]:

train_data,test_data = final_data.randomSplit([0.9,0.1],seed=11)

train_data=train_data.withColumnRenamed('TARGET','label')
test_data=test_data.withColumnRenamed('TARGET','label')

log_reg = LogisticRegression(featuresCol='features',labelCol='label',maxIter=10)
lr_model = log_reg.fit(train_data)



In [45]:
print("final = ",final_data.count())
print("test_data = ", test_data.count())
print("train_data = ", train_data.count())
print("all = ", test_data.count() + train_data.count())

final =  242176
test_data =  24210
train_data =  217966
all =  242176


In [46]:
results = lr_model.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

0.5023973664023248

In [59]:
results.show(10)


+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
+----------+-----+
only showing top 10 rows



In [47]:
'''Param for metric name in evaluation. Supports: - "rmse" (default): root mean squared error - 
"mse": mean squared error - "r2": R^2^ metric - "mae": mean absolute error '''

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

#lr = LinearRegression(maxIter=maxIteration)
#pipeline = Pipeline(stages=[output, log_reg])
pipeline = Pipeline(stages=[log_reg])
#fit_model = pipeline.fit(train_data)
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=log_reg,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=2)

cvModel = crossval.fit(train_data)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
#results.show(10)

In [48]:
cross_results

DataFrame[features: vector, label: int, rawPrediction: vector, probability: vector, prediction: double]

In [49]:
from pyspark.mllib.evaluation import MulticlassMetrics


c = cross_results.rdd
predictionAndLabels = c.map(lambda lp: (lp.prediction, float(lp.label)))
metrics = MulticlassMetrics(predictionAndLabels)
metrics.confusionMatrix()

DenseMatrix(2, 2, [24003.0, 204.0, 2.0, 1.0], 0)

In [50]:
metrics.confusionMatrix().toArray()

array([[2.4003e+04, 2.0000e+00],
       [2.0400e+02, 1.0000e+00]])

In [51]:
print("accuracy = ", metrics.accuracy)
print("recal = ", metrics.recall(1))
print("precision = ", metrics.precision(1))
print("f1 = ", metrics.fMeasure(1.0))



accuracy =  0.9914911193721603
recal =  0.004878048780487805
precision =  0.3333333333333333
f1 =  0.009615384615384616


In [52]:
metrics.weightedPrecision



0.9859989942521602

In [53]:
metrics.weightedFMeasure()

0.9873772286723166

In [54]:
r = results.rdd
r3 = r.collect()

In [56]:
same0 = 0
same1 = 0
sum0 = 0
sum1 = 0
sumall = 0
for i in r3:
    
    if i[1] == 0:
        sum0 += 1
        if i[0] == 0:
            same0 += 1
    elif i[1] == 1:
        sum1 += 1
        if i[0] == 1:
            same1 += 1
    sumall += 1

In [57]:
print('sum0 = ', sum0)
print('sum1 = ', sum1)
print('same0 = ', same0)
print('same1 = ', same1)
print('all = ', sumall)
print('all2 = ', sum0+sum1)

sum0 =  24005
sum1 =  205
same0 =  24003
same1 =  1
all =  24210
all2 =  24210
